In [15]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
from scipy.optimize import root
import math

`DyPy` is a class that we create and store in a file called `MyPy.py`. It contains methods for 

* converting data from a Dynare stucture to Numpy arrays.
* analyzing the data. 

See the file for details. 

In [16]:
from MyPy import DyPy

Creates a file called `t_call.m`. This informs Dynare of the format we want for output. It is put here so that adjusments can be made easily; for example the number of periods can be changed here rather than by looking at the `.mod` file. It is included in the `.mod` file via: `@#include "t_call.m"` in `model_1.mod`

* `order`: the order of approximation for the stochastic simulation. order=1 is a fsrst-order approximation (linearization) Higher-order approximations (e.g., order=2 for second-order) capture nonlinear effects in the model, but they require more computational resources.
* `periods`: number of periods (time steps) for which the simulation will be run. periods=1000 means that the model will be simulated for 1000 periods. The default value is 0, which means no simulation will be performed.
* `nograph`: suppresses the graphical output of the simulation results in Dynare since we draw them separatetly. 
* `irf`: the number of periods for which the impulse response functions (IRFs) will be computed and plotted. 

In [17]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=1000, irf=30);")

 `make_model.ipyn` writes a file called `t_model.m`. `t_model.m`is the `model` block in the Dynare `.mod` file. It can instead be created directlty by commenting out the next `%run` line and coding the model. `make_model.ipyn` simplifies model creation when many of the lines are very similar to each other. Then they can be written in for loops. For the model we are currently working with, there is not much advantage to writing the file via `make_model` but I have done so nonetheless. 
 
 The created file is included in the `.mod` file via `@#include "t_model.m"` in `model_1.mod`. 

In [18]:
#%run make_model_3.ipynb

`make_files.ipynb` writes the other Dynare blocks. See that file for details. However, note that in doing so, it solves for the steady state of the model using `root` from `scipy`. If the results below do not indicate success, the `.mod` file may not run correctly. Also, note that though we solve for the steady state here, we do not give Dynare the steady state but rather use these as starting values for their ss solver. I will later code a verification the steady states are the same. 

In [19]:
%run make_files_3.ipynb

eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2
eps,tht,alf,bet,om_0,om_1,om_2,delt,var_a_0,var_a_1,var_a_2,rho_a_0,rho_a_1,rho_a_2
 message: The solution converged.
 success: True
  status: 1
     fun: [-3.796e-12 -3.796e-12 -4.282e-13  0.000e+00 -6.417e-13
            3.003e-13  3.008e-13  0.000e+00  0.000e+00]
       x: [ 3.718e-01  3.141e-01  3.141e-01  1.331e+00  1.331e+00
            1.331e+00  3.624e-01  3.624e-01  3.624e-01]
    nfev: 26
    fjac: [[-3.071e-06 -3.071e-06 ...  1.019e-16  2.593e-16]
           [-4.169e-06 -4.154e-06 ...  1.416e-16  3.581e-16]
           ...
           [ 6.056e-01 -6.388e-01 ... -2.834e-01  3.388e-01]
           [ 2.046e-01 -2.241e-02 ... -2.085e-01 -9.502e-02]]
       r: [ 1.422e+00  7.316e-01 ...  1.220e+00  4.605e-01]
     qtf: [-1.276e-12 -4.305e-12  2.114e-12  6.153e-12 -7.852e-13
           -5.714e-12 -3.405e-12  3.354e-12 -1.839e-11]


Tells octave where to find the .m file it will run. The file is `call_dynare`. Set to wherever you have this file. 




In [20]:
#a=octave.addpath(os.path.expanduser('C:/Users/blankenw/GitHub/dynare_progs/np_1'))

If you see the following error message:    
`warning: function C:\dynare\6.1\matlab\+identification\display.m shadows a built-in function`
it can be ignored

Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [21]:
y=1
x=octave.call_dynare(y,nout=1)



Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
4 block(s) found:
  3 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 9 equation(s)
                                 and 9 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 9 equation(s)
                                 and 9 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m00s.

STEADY-STATE RESULTS:

n_0 		 0.324351
n_1 		 0.337825
n_2 		 0.337824
k_0 		 1.01326
k_1 		 0.79405
k_2 		 0.79405
c_0 		 0.283236
c_1

Oct2PyError: Octave evaluation error:
error: Blanchard & Kahn conditions are not satisfied: indeterminacy.
error: called from:
    print_info at line 33, column 5
    stoch_simul at line 119, column 5
    driver at line 517, column 27
    dynare at line 310, column 5
    call_dynare at line 5, column 4

Dynare creates a `structure` which we will need so we `pull` it.

The output from Dynare is stored in a file called `oo_`. This brings `oo_` in as `oo`.

In [ ]:
oo = octave.pull('oo_') 
type(oo)

Create an instance of DyPy.

In [ ]:
t1 = DyPy(oo)


Draw all impulse response functions. 

In [ ]:
for shock in {key[-1] for key in t1.dict_irfs.keys()}:
    t1.plot_irfs(shock,cols=3)

The rest of this just gives insights regarding the 'oo' object. 

In [ ]:
print(type(oo))
print(oo.keys())
print(oo['irfs'].keys())
print(type(oo['n_0_a_0_shock']))
oo['n_0_a_0_shock']

Gives a list of all the vaues that are numpy arrays and lists in dictionary `oo`.

In [ ]:
numpy_keys = [key for key, value in oo.items() if isinstance(value, np.ndarray)]
list_keys = [key for key, value in oo.items() if isinstance(value, list)]
print(numpy_keys)
print(list_keys)

In [ ]:
type(t1.oo_dict)

In [ ]:
key_type_list = [(key, type(value)) for key, value in oo.items()]
print(key_type_list)
key_type_list = [(key, type(value)) for key, value in t1.oo_dict.items()]
print(key_type_list)

In [ ]:
attributes = [attr for attr in dir(t1) if not attr.startswith('__')]
for attr in attributes:
    print(attr)

In [ ]:
t1.steady_state

In [ ]:
oo